In [2]:
import pandas as pd
import psycopg2

In [2]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [3]:
DBNAME = "opportunity_youth"

In [4]:
conn = psycopg2.connect(dbname=DBNAME)
cursor = conn.cursor()
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
tables = []
for table in cursor.fetchall():
    tables.append(table[0])
tables

['pums_2017',
 'puma_names_2010',
 'wa_jobs_2017',
 'wa_geo_xwalk',
 'ct_puma_xwalk']

In [5]:
pums_2017_df = pd.read_sql("SELECT * FROM pums_2017;", conn)
puma_names_2010_df = pd.read_sql("SELECT * FROM puma_names_2010;", conn)
wa_jobs_2017_df = pd.read_sql("SELECT * FROM wa_jobs_2017;", conn)
wa_geo_xwalk_df = pd.read_sql("SELECT * FROM wa_geo_xwalk;", conn)
ct_puma_xwalk_df = pd.read_sql("SELECT * FROM ct_puma_xwalk;", conn)

In [5]:
df_new = pd.read_sql("""
SELECT sporder, puma, agep, cow, wkl, sch, schl, rt
FROM pums_2017
WHERE (agep BETWEEN 16.0 AND 24.9)
AND CAST(sch AS int) = 1
AND CAST(cow AS int) = 9
""", conn)
df_new


,sporder,puma,agep,cow,wkl,sch,schl,rt
0,02,10502,23.0,9,3,1,13,P
1,02,11503,24.0,9,3,1,22,P
2,02,10502,19.0,9,3,1,16,P
3,03,11615,22.0,9,3,1,21,P
4,03,10400,19.0,9,3,1,16,P
...,...,...,...,...,...,...,...,...
269,01,11605,23.0,9,3,1,19,P
270,02,10503,18.0,9,3,1,16,P
271,04,11103,23.0,9,3,1,21,P
272,06,11505,21.0,9,3,1,13,P


In [12]:
pums_2017_df.shape

(359075, 286)

In [13]:
print(pums_2017_df.cow.dtype)

object


In [14]:
df_new.info

<bound method DataFrame.info of     sporder   puma  agep cow wkl sch schl rt
0        02  10502  23.0   9   3   1   13  P
1        02  10502  19.0   9   3   1   16  P
2        02  11503  24.0   9   3   1   22  P
3        02  11503  19.0   9   3   1   16  P
4        03  10400  19.0   9   3   1   16  P
..      ...    ...   ...  ..  ..  ..  ... ..
269      01  11605  23.0   9   3   1   19  P
270      04  11502  20.0   9   3   1   16  P
271      04  11103  23.0   9   3   1   21  P
272      06  11505  21.0   9   3   1   13  P
273      02  11608  21.0   9   3   1   21  P

[274 rows x 8 columns]>

In [15]:
df_new.cow.isna().sum()

0

In [16]:
#list of Employment Status (COW), School Status(SCH), NWAV (Availble for work), NWAB (Temporary absence)
#NWLA (On lay off), NLWK (Looking for work).
#Table to view NULL values, to allow us to add filters to include more people.
df_cs = pd.read_sql("""
SELECT cow, sch, nwav, nwab, nwla, nwlk
FROM pums_2017
;""", conn)
df_cs

,cow,sch,nwav,nwab,nwla,nwlk
0,None,1,5,2,2,2
1,None,1,5,2,2,2
2,None,1,5,2,2,2
3,None,1,5,2,2,2
4,1,1,5,3,3,3
...,...,...,...,...,...,...
359070,5,1,5,3,3,3
359071,1,3,5,3,3,3
359072,1,2,5,2,2,2
359073,None,1,5,2,2,2


In [17]:
df_cs.describe()

,cow,sch,nwav,nwab,nwla,nwlk
count,215545,347443,291865,291865,291865,291865
unique,9,3,5,3,3,3
top,1,1,5,3,3,3
freq,132602,264532,266800,172064,166563,172483


In [32]:
puma_test = pd.read_sql(
""" SELECT puma, puma_name
FROM puma_names_2010
WHERE state_name LIKE 'Washington%' AND puma_name LIKE 'King County%'
""", conn)
puma_test

,puma,puma_name
0,11606,"King County (Northwest)--Shoreline, Kenmore & ..."
1,11607,"King County (Northwest)--Redmond, Kirkland Cit..."
2,11608,King County (Northwest Central)--Greater Belle...
3,11609,"King County (Central)--Sammamish, Issaquah, Me..."
4,11610,"King County (Central)--Renton City, Fairwood, ..."
5,11613,King County (Southwest Central)--Kent City ...
6,11614,King County (Southwest)--Auburn City & Lakelan...
7,11615,"King County (Southeast)--Maple Valley, Covingt..."
8,11616,"King County (Northeast)--Snoqualmie City, Cott..."
9,11611,"King County (West Central)--Burien, SeaTac, Tu..."


In [49]:
puma_test.describe()

,puma,puma_name
count,11,11
unique,11,11
top,11613,"King County (Far Southwest)--Federal Way, Des ..."
freq,1,1


In [53]:
# List of total pop in king county between 16 and 24.9
total_kids = pd.read_sql("""
SELECT A.puma, agep
FROM pums_2017 A
JOIN puma_names_2010 B ON A.puma = B.puma
WHERE state_name LIKE 'Washington%' AND puma_name LIKE 'King County%'
AND (agep BETWEEN 16.0 AND 24.9)
""", conn)
Not_oy

,puma,agep
0,11606,21.0
1,11606,21.0
2,11608,18.0
3,11610,24.0
4,11610,24.0
...,...,...
6193,11607,21.0
6194,11606,18.0
6195,11606,18.0
6196,11607,21.0


In [55]:
# List of working kids without a HS diploma or GED
#WIP WORKING LIST
working_wodip = pd.read_sql ("""
SELECT A.puma, agep, schl
FROM pums_2017 A
JOIN puma_names_2010 B ON A.puma = B.puma
WHERE state_name LIKE 'Washington%' AND puma_name LIKE 'King County%'
AND (agep BETWEEN 16.0 AND 24.9)
""", conn)
working_wodip

,puma,agep,schl
0,11606,21.0,19
1,11606,21.0,16
2,11608,18.0,18
3,11610,24.0,21
4,11610,24.0,21
...,...,...,...
6193,11607,21.0,16
6194,11613,23.0,21
6195,11606,18.0,18
6196,11607,19.0,16


In [8]:
#df_cs, but with filters. Having issues with NWAB/NWLA/NWLK setting them to = 1 OR 3
#Kids between 16 and 24.9 in King County that are not working

puma_df_piv = pd.read_sql("""
SELECT A.puma, sporder, serialno, sch, schl, esr, cow, agep
FROM pums_2017 A
JOIN puma_names_2010 B ON A.puma = B.puma
WHERE state_name LIKE 'Washington%' AND puma_name LIKE 'King County%'
AND (agep BETWEEN 16.0 AND 24.9)
""", conn)
puma_df_piv
#OR CAST(wkexrel as int) = (7 OR 8 OR 9 OR 12 OR 15)
#JOIN puma_names_2010 B ON A.puma = B.puma
#OR CAST(nwab as int) = 3
#OR CAST(nwla as int) = 3
#OR CAST(nwlk as int) = 3)

,puma,sporder,serialno,sch,schl,esr,cow,agep
0,11606,01,2013000003218,3,19,1,1,21.0
1,11606,02,2013000003218,1,16,1,1,21.0
2,11608,03,2013000003370,2,18,6,3,18.0
3,11610,01,2013000003570,1,21,1,1,24.0
4,11610,02,2013000003570,1,21,1,1,24.0
...,...,...,...,...,...,...,...,...
6193,11613,01,2017001518359,1,11,6,1,18.0
6194,11607,01,2017001520559,3,20,1,1,21.0
6195,11606,01,2017001526134,2,18,3,1,18.0
6196,11613,01,2017001530818,1,21,1,1,23.0


In [64]:
df_cs_fil.describe()

,cow,nwav,nwab,nwla,nwlk,puma
count,3185,3437,3437,3437,3437,3437
unique,9,4,3,3,3,11
top,1,5,3,3,3,11614
freq,2569,3259,3204,3122,3232,376


In [46]:
#NAN Count for df_cs_fil
len(df_cs_fil) - df_cs_fil.count()

cow     252
nwav      0
nwab      0
nwla      0
nwlk      0
puma      0
dtype: int64

In [19]:
puma_df_piv

,puma,sporder,serialno,sch,schl,esr,cow,agep
0,11606,01,2013000003218,3,19,1,1,21.0
1,11606,02,2013000003218,1,16,1,1,21.0
2,11608,03,2013000003370,2,18,6,3,18.0
3,11610,01,2013000003570,1,21,1,1,24.0
4,11610,02,2013000003570,1,21,1,1,24.0
...,...,...,...,...,...,...,...,...
6193,11613,01,2017001518359,1,11,6,1,18.0
6194,11607,01,2017001520559,3,20,1,1,21.0
6195,11606,01,2017001526134,2,18,3,1,18.0
6196,11613,01,2017001530818,1,21,1,1,23.0


In [69]:
puma_df_piv.loc[(puma_df_piv > 15.9 & puma_df_piv < 18.1), "AgeGroup"] = "16 - 18"

In [25]:
#true or false for age groups
puma_df_piv["group_one"] = puma_df_piv.agep.between(15.9, 18.9)
puma_df_piv["group_two"] = puma_df_piv.agep.between(19.0, 21.9)
puma_df_piv["group_three"] = puma_df_piv.agep.between(22.0, 24.9)

In [67]:
# for x in puma_df_piv.group_one:
#     if x is True:
#         puma_df_piv["AgeGroup"] = "16-18"
#     else puma_df_piv["AgeGroup"] = "0"
#     elif x in puma_df_piv.group_two:
#         if x is True:
#             puma_df_piv["AgeGroup"] = "19-21"
#     elif x in puma_df_piv.group_three:
#         if x is True:
#             puma_df_piv["AgeGroup"] = "22-24"

df_puma_test = puma_df_piv[puma_df_piv["group_one"]]["AgeGroup"] = "16-18"
# puma_df_piv[puma_df_piv.loc["group_two"]]['AgeGroup'] = "19-21"
# puma_df_piv[puma_df_piv.loc["group_three"]]['AgeGroup'] = "22-24"

In [68]:
puma_df_piv[puma_df_piv.loc["group_one"]]

In [56]:
puma_df_piv

,puma,sporder,serialno,sch,schl,esr,cow,agep,group_one,group_two,group_three,AgeGroup
0,11606,01,2013000003218,3,19,1,1,21.0,False,True,False,16-18
1,11606,02,2013000003218,1,16,1,1,21.0,False,True,False,16-18
2,11608,03,2013000003370,2,18,6,3,18.0,True,False,False,16-18
3,11610,01,2013000003570,1,21,1,1,24.0,False,False,True,16-18
4,11610,02,2013000003570,1,21,1,1,24.0,False,False,True,16-18
...,...,...,...,...,...,...,...,...,...,...,...,...
6193,11613,01,2017001518359,1,11,6,1,18.0,True,False,False,16-18
6194,11607,01,2017001520559,3,20,1,1,21.0,False,True,False,16-18
6195,11606,01,2017001526134,2,18,3,1,18.0,True,False,False,16-18
6196,11613,01,2017001530818,1,21,1,1,23.0,False,False,True,16-18
